# Sistema de Seguridad Multimodal Inteligente

Este notebook implementa un sistema de seguridad avanzado capaz de detectar personas y reconocer rostros en videos utilizando técnicas de **Visión por Computador Ultra-Ligeras**.

### 🚀 Características Principales
- **Detección Ultra-Ligera**: Usa Haar Cascades para funcionar en dispositivos de bajos recursos (Raspberry Pi).
- **Reconocimiento Geométrico**: Identifica personas basándose en ratios faciales únicos (distancia ojos, nariz, boca) sin necesidad de redes neuronales pesadas.
- **Tracking Inteligente**: Sistema de estados para evitar alertas repetitivas mientras una persona permanece en cámara.

### 🛠️ Tecnologías
- **OpenCV**: Procesamiento de imagen y detección.
- **Python**: Lógica del sistema.
- **Matplotlib**: Visualización de resultados.

## 1. Configuración e Instalación
Instalamos las librerías necesarias.

In [1]:
!pip install opencv-python matplotlib numpy

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Optional, List, Dict
import time

# Configuración para mostrar imágenes en el notebook
def show_frame(frame, title="Frame"):
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

## 2. Módulo de Detección (LightweightDetector)
Este módulo utiliza **Haar Cascades** de OpenCV para detectar rostros y ojos. Es extremadamente rápido y eficiente.

In [3]:
@dataclass
class Detection:
    """Estructura de datos para una detección."""
    x: int
    y: int
    width: int
    height: int
    confidence: float = 1.0
    landmarks: Optional[dict] = None

class LightweightDetector:
    """
    Detector ultra-ligero usando Haar Cascades.
    Optimizado para rendimiento en CPU.
    """
    def __init__(self):
        # Cargar clasificadores pre-entrenados
        # Nota: En Colab/Local puede requerir rutas absolutas si no encuentra los archivos xml
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        
    def detect(self, frame: np.ndarray) -> List[Detection]:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detectar caras
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )
        
        detections = []
        for (x, y, w, h) in faces:
            # Buscar ojos dentro de la región de la cara
            roi_gray = gray[y:y+h, x:x+w]
            eyes = self.eye_cascade.detectMultiScale(roi_gray, 1.1, 3)
            
            landmarks = None
            # Si encontramos 2 ojos, podemos estimar la geometría
            if len(eyes) >= 2:
                # Ordenar ojos por posición X
                eyes = sorted(eyes, key=lambda e: e[0])
                left_eye = (x + eyes[0][0] + eyes[0][2]//2, y + eyes[0][1] + eyes[0][3]//2)
                right_eye = (x + eyes[1][0] + eyes[1][2]//2, y + eyes[1][1] + eyes[1][3]//2)
                
                # Estimar boca y nariz basado en proporciones antropométricas estándar
                mouth_center = (x + w//2, y + int(h * 0.75))
                nose_center = (x + w//2, y + int(h * 0.55))
                chin_center = (x + w//2, y + h)
                
                landmarks = {
                    "left_eye": left_eye,
                    "right_eye": right_eye,
                    "mouth": mouth_center,
                    "nose": nose_center,
                    "chin": chin_center
                }
            
            detections.append(Detection(x, y, w, h, 1.0, landmarks))
            
        return detections

## 3. Módulo de Reconocimiento (GeometricRecognizer)
Este módulo extrae "firmas geométricas" de los rostros. A diferencia de los embeddings de Deep Learning, esto usa matemáticas simples sobre las distancias entre puntos clave.

In [4]:
class GeometricRecognizer:
    """
    Reconocimiento facial basado en ratios geométricos.
    """
    def __init__(self, tolerance: float = 0.15):
        self.tolerance = tolerance
        self.known_profiles = {}  # {id: {"name": str, "ratios": list}}
    
    def extract_ratios(self, landmarks: dict) -> Optional[List[float]]:
        """Extrae 4 ratios geométricos únicos del rostro."""
        if not landmarks:
            return None
            
        le = np.array(landmarks["left_eye"])
        re = np.array(landmarks["right_eye"])
        mo = np.array(landmarks["mouth"])
        ch = np.array(landmarks["chin"])
        no = np.array(landmarks["nose"])
        
        # Distancias clave
        eye_dist = np.linalg.norm(re - le)
        eyes_center = (le + re) / 2
        eye_mouth_dist = np.linalg.norm(mo - eyes_center)
        nose_chin_dist = np.linalg.norm(ch - no)
        face_height = np.linalg.norm(ch - eyes_center)
        
        if eye_dist == 0 or face_height == 0: return None
        
        # Ratios invariantes a la escala (distancia / tamaño cara)
        return [
            eye_dist / face_height,          # Ancho ojos vs Alto cara
            eye_mouth_dist / face_height,    # Ojos-Boca vs Alto cara
            nose_chin_dist / face_height,    # Nariz-Barbilla vs Alto cara
            eye_dist / eye_mouth_dist        # Relación triángulo facial
        ]

    def register_person(self, name: str, ratios: List[float]):
        """Guarda una nueva persona conocida."""
        pid = len(self.known_profiles) + 1
        self.known_profiles[pid] = {"name": name, "ratios": ratios}
        print(f"✅ Persona registrada: {name}")

    def identify(self, ratios: List[float]) -> str:
        """Identifica a una persona comparando ratios."""
        if not ratios: return "Desconocido"
        
        best_match = "Desconocido"
        best_score = float('inf')
        
        for pid, profile in self.known_profiles.items():
            known_ratios = profile["ratios"]
            # Diferencia promedio (Distancia L1)
            diff = np.mean([abs(a - b) for a, b in zip(ratios, known_ratios)])
            
            if diff < self.tolerance and diff < best_score:
                best_score = diff
                best_match = profile["name"]
                
        return best_match

## 4. Lógica de Tracking Inteligente
Para evitar que el sistema envíe alertas constantes (spam) cuando una persona está parada frente a la cámara, implementamos una máquina de estados simple.

In [5]:
class SecuritySystem:
    def __init__(self):
        self.detector = LightweightDetector()
        self.recognizer = GeometricRecognizer()
        
        # Estado del tracking
        self.person_in_view = False
        self.frames_without_person = 0
        self.reset_threshold = 10  # Frames para considerar que la persona se fue
        self.alerts_log = []

    def process_frame(self, frame):
        detections = self.detector.detect(frame)
        annotated_frame = frame.copy()
        
        status_message = ""
        
        if detections:
            # Persona detectada
            self.frames_without_person = 0
            det = detections[0]  # Procesamos la primera cara encontrada
            
            # Dibujar bounding box
            cv2.rectangle(annotated_frame, (det.x, det.y), 
                         (det.x+det.width, det.y+det.height), (0, 255, 0), 2)
            
            # Dibujar landmarks
            if det.landmarks:
                for point in det.landmarks.values():
                    cv2.circle(annotated_frame, point, 3, (255, 0, 0), -1)
                
                # Intentar reconocimiento
                ratios = self.recognizer.extract_ratios(det.landmarks)
                name = self.recognizer.identify(ratios)
                
                # Etiqueta
                cv2.putText(annotated_frame, name, (det.x, det.y-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
                # Lógica de Alerta Inteligente
                if not self.person_in_view:
                    self.person_in_view = True
                    alert = f"🚨 ALERTA: {name} detectado"
                    self.alerts_log.append(alert)
                    status_message = alert
                    # Auto-registro para demo (si es desconocido y tiene landmarks claros)
                    if name == "Desconocido" and ratios:
                        self.recognizer.register_person("Visitante_1", ratios)
                        status_message += " -> Registrado como Visitante_1"
            
        else:
            # Nadie en cámara
            self.frames_without_person += 1
            if self.frames_without_person >= self.reset_threshold:
                if self.person_in_view:
                    self.alerts_log.append("ℹ️ Persona salió de cámara")
                self.person_in_view = False
        
        # Mostrar estado en el frame
        if status_message:
            cv2.putText(annotated_frame, status_message, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        elif self.person_in_view:
            cv2.putText(annotated_frame, "Tracking activo...", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            
        return annotated_frame

## 5. Ejecución con Video
Cargamos un video, lo procesamos frame a frame y guardamos el resultado.

In [6]:
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error al abrir el video")
        return
    
    # Propiedades del video
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Configurar escritor de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    system = SecuritySystem()
    frame_count = 0
    
    print("Procesando video...")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Procesar frame
        processed_frame = system.process_frame(frame)
        
        # Guardar
        out.write(processed_frame)
        
        # Mostrar progreso cada 30 frames
        if frame_count % 30 == 0:
            print(f".", end="")
        frame_count += 1
        
    cap.release()
    out.release()
    print(f"\n¡Listo! Video guardado en {output_path}")
    print("\n📜 Log de Alertas:")
    for log in system.alerts_log:
        print(log)

# --- EJECUCIÓN ---
# Para probar, puedes subir un video llamado 'test_video.mp4' o usar la webcam en local
# process_video('test_video.mp4', 'output_video.mp4')

## 6. Prueba Rápida (Imagen Estática)
Si no tienes un video a mano, probemos el sistema con una imagen generada sintéticamente o cargada.

In [ ]:
# Crear una imagen de prueba simple (o cargar una real)
# Aquí simulamos un frame vacío para demostrar que no crashea
dummy_frame = np.zeros((480, 640, 3), dtype=np.uint8)
cv2.putText(dummy_frame, "Frame de Prueba (Sin Cara)", (150, 240), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

system = SecuritySystem()
result = system.process_frame(dummy_frame)
show_frame(result, "Resultado Prueba")

print("Nota: Para ver detecciones reales, sube una imagen con una cara y usa:")
print("img = cv2.imread('tu_foto.jpg')")
print("res = system.process_frame(img)")
print("show_frame(res)")